In [184]:
import pandas as pd # library for data analsysis
import requests # library to handle requests
import folium   #map rendering librar
import numpy as np # library to handle data in a vectorized manner
!pip install folium

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

print('Library completed.')
print ('GO Coursera!')

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Library completed.
GO Coursera!


In [185]:
#Connect with the API Foursquare
CLIENT_ID = '4XTRV3HE4JKVXI4MTVIU0TITWGXNE0ALJOAD51SK32CWM0Y2' #Foursquare ID
CLIENT_SECRET = 'UOKZKX0WVA2AQ2FCFATDDTIMMQZI2VPZQCYKBUTKKBERKDVX'  #Foursquare Secret


VERSION = '20200427' # Foursquare API version

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Credentails:
CLIENT_ID: 4XTRV3HE4JKVXI4MTVIU0TITWGXNE0ALJOAD51SK32CWM0Y2


In [186]:
# City adds
LIMIT = 500 # Maximum is 100 per city
cities = ['Maracaibo, Estado Zulia', 'Caracas, Distrito Federal', 'Los Teques, Estado Miranda', 'Valencia, Carabobo', 'Maracay, Estado Aragua']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d196941735") # HOSPITAL CATEGORY ID ON FOURSQUARE
    results[city] = requests.get(url).json()

In [187]:
#Locations
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [188]:
#Maps marks
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Number of Hospital in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Number of Hospital in Maracaibo, Estado Zulia =  45
Showing Top 100
Number of Hospital in Caracas, Distrito Federal =  70
Showing Top 100
Number of Hospital in Los Teques, Estado Miranda =  5
Showing Top 100
Number of Hospital in Valencia, Carabobo =  12
Showing Top 100
Number of Hospital in Maracay, Estado Aragua =  16
Showing Top 100


In [189]:
print (cities[0])
maps[cities[0]]

Maracaibo, Estado Zulia


In [190]:
print (cities[1])
maps[cities[1]]

Caracas, Distrito Federal


In [191]:
print (cities[2])
maps[cities[2]]

Los Teques, Estado Miranda


In [192]:
print (cities[3])
maps[cities[3]]

Valencia, Carabobo


In [193]:
print (cities[4])
maps[cities[4]]

Maracay, Estado Aragua


In [194]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Maracaibo, Estado Zulia
Mean Distance from Mean coordinates
0.03188783424502879
Caracas, Distrito Federal
Mean Distance from Mean coordinates
0.04051426513138617
Los Teques, Estado Miranda
Mean Distance from Mean coordinates
0.02741244002207569
Valencia, Carabobo
Mean Distance from Mean coordinates
0.016359083636850607
Maracay, Estado Aragua
Mean Distance from Mean coordinates
0.022382394798906993


In [195]:
print (cities[0])
maps[cities[0]]

Maracaibo, Estado Zulia


In [196]:
print (cities[1])
maps[cities[1]]

Caracas, Distrito Federal


In [197]:
print (cities[2])
maps[cities[2]]

Los Teques, Estado Miranda


In [198]:
print (cities[3])
maps[cities[3]]

Valencia, Carabobo


In [199]:
print (cities[4])
maps[cities[4]]

Maracay, Estado Aragua


In [200]:
city = 'Valencia, Carabobo'
venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 

print(city)
print("Mean Distance from Mean coordinates")
dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)
dists.sort()
print(np.mean(dists[:-1]))# Distance more than 1 dont show

Valencia, Carabobo
Mean Distance from Mean coordinates
0.01448824022257683


# EdgarGarcia 